In [ ]:
# Hey Chris, in the next line put the folder where all the student folders are contained, I think I've left it as you had it but best to make sure.
text_analysis_folders_path = '/content/drive/MyDrive/Student Text analysis'
# Here put the link that you get when you press copy link on the database spreadsheet. This definitely needs changing as it is currently connected to my database
database_path = 'https://docs.google.com/spreadsheets/d/1Y4BzFlHPEMYWtK1V8wtOA4WbCEG4Bb_7wTD0QM4XlI0/edit?usp=sharing'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
import pandas as pd
import numpy as np
import os
!pip install xlsxwriter
from openpyxl import load_workbook
from sklearn.linear_model import LinearRegression

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 149 kB 5.1 MB/s 


In [ ]:
list_of_folders = []
file_paths = []

for i in os.listdir(text_analysis_folders_path):
  try:
    os.listdir(text_analysis_folders_path+"/"+i)
    list_of_folders.append(text_analysis_folders_path+"/"+i)
  except:
    print('not a folder')

for j in list_of_folders:
  for filename in os.listdir(j):
        file_paths.append(j+"/"+filename)

not a folder
not a folder


In [ ]:
def file_name(path):
  last_slash = path.rfind('/')
  ftype = path.find('.gsheet')
  file_name = path[last_slash+1:ftype]
  return file_name

def user_details(file_name):
  list_of_details = file_name.split('-')
  user_id = list_of_details[0].strip()
  name = list_of_details[1].strip()
  date = '{}/{}/{}'.format(
      list_of_details[2],
      list_of_details[3],
      list_of_details[4]
  )
  return user_id, name, date

def gsheet2df(gsheet):
    """ Converts Google sheet data to a Pandas DataFrame.
    Note: This script assumes that your data contains a header file on the first row!

    Also note that the Google API returns 'none' from empty cells - in order for the code
    below to work, you'll need to make sure your sheet doesn't contain empty cells,
    or update the code to account for such instances.

    """
    header = gsheet.get('values', [])[0]   # Assumes first line is header!
    values = gsheet.get('values', [])[1:]  # Everything else is data.
    if not values:
        print('No data found.')
    else:
        all_data = []
        for col_id, col_name in enumerate(header):
            column_data = []
            for row in values:
                column_data.append(row[col_id])
            ds = pd.Series(data=column_data, name=col_name)
            all_data.append(ds)
        df = pd.concat(all_data, axis=1)
        return df

In [ ]:
def read_csv(path):
  df = pd.read_csv(path)
  return df

def read_gsheet(path):
  path = file_name(path)
  values = gc.open(path).get_worksheet(0).get_all_values()
  df = pd.DataFrame(values[1:], columns=values[0])
  return df 

def path_checker(path):
  if path.endswith('.csv'):
    df = read_csv(path)
  elif path.endswith('.gsheet'):
    df = read_gsheet(path)
  else:
    print('not a proper filetype')
  return df

In [ ]:
info = {}


for path in file_paths:
  individual_file_name = file_name(path)
  id, name, date = user_details(individual_file_name)

  if 'Student ID' in info:
      info['Student ID'].append(id)
  else:
      info['Student ID'] = [id]
    
  if 'Student Name' in info:
      info['Student Name'].append(name)
  else:
      info['Student Name'] = [name]

  if 'Lesson Date' in info:
      info['Lesson Date'].append(date)
  else:
      info['Lesson Date'] = [date]



  df = path_checker(path)
  keys = df['Data'].values
  values = df['Document 1'].values
  for i,j in zip(keys, values):   
    if i in info:
        info[i].append(j)
    else:
        info[i] = [j]

In [ ]:
columns_wanted = ['Student Name',
                  'Student ID',
                  'Lesson Date',
                  'Average Sentence Length',
                  'Lexical diversity (VOCD)',
                  'Lexical diversity (MTLD)',
                  'Spelling errors total',
                  'A1 type %',
                  'A2 type %',
                  'B1 type %',
                  'B2 type %',
                  'C1 type %',
                  'C2 type %',
                  'Announce Goals type count',
                  'Attitude marker type count',
                  'Code gloss type count',
                  'Emphatic type count',
                  'Endophoric type count',
                  'Evidential type count',
                  'Hedge type count',
                  'Label stage type count',
                  'Logical connective type count',
                  'Person marker type count',
                  'Relational marker type count',
                  'Sequencing type count',
                  'Topic shift type count',
                  'CEFR level',
                  'Flesch Kincaid Reading Grade - CEFR level']


CEFR_level_converter = {'C2':1,
                        'C2+':2,
                        'C1':3,
                        'C1+':4,
                        'B2':5,
                        'B2+':6,
                        'B1':7,
                        'B1+':8,
                        'A2':9,
                        'A2+':10,
                        'A1':11,
                        'A1+':12}               

filtered_dict = {key: value for key, value in info.items() if key in columns_wanted}     

In [ ]:
df = pd.DataFrame.from_dict(filtered_dict)
lesson_number_dfs = []

# Join this df to the main df

for name in df['Student Name'].unique():
  lesson_date_list = []
  lesson_number_list = []
  lesson_dates = sorted(df[df['Student Name']==name]['Lesson Date'].unique())
  for lesson_number, lesson_date in enumerate(lesson_dates):
    lesson_date_list = []
    lesson_number_list = []
    lesson_date_list.append(lesson_date)
    lesson_number_list.append(lesson_number+1)
    loop_df = pd.DataFrame(list(zip(lesson_date_list, lesson_number_list)),
        columns = ['Lesson Date', 'Lesson Number'])
    loop_df['Student Name'] = name
    lesson_number_dfs.append(loop_df)

lesson_number_df = pd.concat(lesson_number_dfs)
df = pd.merge(df,lesson_number_df, how='left', on=['Student Name', 'Lesson Date'])

In [ ]:
df['Total Lessons'] = len(df)/len(df['Student Name'].unique())
df['Lesson Date'] = pd.to_datetime(df['Lesson Date'])
df['CEFR numeric'] = df['CEFR level'].map(CEFR_level_converter)
df['Flesch Kincaid numeric'] = df['Flesch Kincaid Reading Grade - CEFR level'].map(CEFR_level_converter)


In [ ]:
lexis_evp_type = df[[
    'Student ID',
    'Lesson Number',
    'A1 type %',
    'A2 type %',
    'B1 type %',
    'B2 type %',
    'C1 type %',
    'C2 type %'
]].melt(['Student ID', 'Lesson Number'])

In [ ]:
lexis_evp_type

,Student ID,Lesson Number,variable,value
0,0002,27,A1 type %,42.31
1,0002,13,A1 type %,49.44
2,0002,26,A1 type %,35.15
3,0002,12,A1 type %,48.08
4,0002,3,A1 type %,49.34
...,...,...,...,...
265,0001,2,C2 type %,1.03
266,0001,4,C2 type %,0.00
267,0001,5,C2 type %,0.81
268,0001,13,C2 type %,0.00


In [ ]:
meta_discourse = df[[
                  'Student ID',
                  'Lesson Number',
                  'Announce Goals type count',
                  'Attitude marker type count',
                  'Code gloss type count',
                  'Emphatic type count',
                  'Endophoric type count',
                  'Evidential type count',
                  'Hedge type count',
                  'Label stage type count',
                  'Logical connective type count',
                  'Person marker type count',
                  'Relational marker type count',
                  'Sequencing type count',
                  'Topic shift type count'
]].melt(['Student ID', 'Lesson Number'])
meta_discourse

,Student ID,Lesson Number,variable,value
0,0002,27,Announce Goals type count,0.0
1,0002,13,Announce Goals type count,0.0
2,0002,26,Announce Goals type count,0.0
3,0002,12,Announce Goals type count,0.0
4,0002,3,Announce Goals type count,0.0
...,...,...,...,...
580,0001,2,Topic shift type count,0.0
581,0001,4,Topic shift type count,0.0
582,0001,5,Topic shift type count,1.0
583,0001,13,Topic shift type count,0.0


In [ ]:
model = LinearRegression()
test_df = df[df['Student Name']=='Daisuke']
model.fit(test_df['Lesson Number'].values.reshape(len(test_df),1),test_df['Lexical diversity (MTLD)'].values.reshape(len(test_df),1))
model.coef_

array([[-1.27010714]])

In [ ]:
test_df.columns

Index(['Student ID', 'Student Name', 'Lesson Date', 'Average Sentence Length',
       'Lexical diversity (VOCD)', 'Lexical diversity (MTLD)',
       'Spelling errors total', 'A1 type %', 'A2 type %', 'B1 type %',
       'B2 type %', 'C1 type %', 'C2 type %', 'Announce Goals type count',
       'Attitude marker type count', 'Code gloss type count',
       'Emphatic type count', 'Endophoric type count', 'Evidential type count',
       'Hedge type count', 'Label stage type count',
       'Logical connective type count', 'Person marker type count',
       'Relational marker type count', 'Sequencing type count',
       'Topic shift type count', 'CEFR level',
       'Flesch Kincaid Reading Grade - CEFR level', 'Lesson Number',
       'Total Lessons', 'CEFR numeric', 'Flesch Kincaid numeric'],
      dtype='object')

In [ ]:
for name in df['Student Name'].unique():
  sorted_df = sorted(df[df['Student Name']==name]['Lesson Number'])

In [ ]:
sh = gc.open_by_url(database_path)
worksheet = sh.worksheet('Database')
worksheet.clear()
worksheet.update([df.columns.astype(str).values.tolist()] + df.values.astype(str).tolist())
worksheet = sh.worksheet('Lexis EVP Type')
worksheet.clear()
worksheet.update([lexis_evp_type.columns.astype(str).values.tolist()] + lexis_evp_type.values.astype(str).tolist())
worksheet = sh.worksheet('Meta Discourse')
worksheet.clear()
worksheet.update([meta_discourse.columns.astype(str).values.tolist()] + meta_discourse.values.astype(str).tolist())

{'spreadsheetId': '1Y4BzFlHPEMYWtK1V8wtOA4WbCEG4Bb_7wTD0QM4XlI0',
 'updatedRange': "'Meta Discourse'!A1:D586",
 'updatedRows': 586,
 'updatedColumns': 4,
 'updatedCells': 2344}